In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np, faiss, json

# загрузка той же модели, что использовалась при индексации
model = SentenceTransformer("intfloat/multilingual-e5-base")

# два индекса: по заголовкам и по телу
idx_title = faiss.read_index("smartwop_title.faiss")
idx_body  = faiss.read_index("smartwop_body.faiss")

# метаданные [{id, title, body}, ...]
meta = json.load(open("smartwop_docs_meta.json", "r", encoding="utf-8"))

def _reconstruct_batch(index: faiss.Index, ids):
    """Достаём исходные эмбеддинги по списку id (IndexFlatIP поддерживает reconstruct)."""
    vecs = [index.reconstruct(int(i)) for i in ids]
    return np.asarray(vecs, dtype="float32")

def search(query: str, k: int = 5, alpha: float = 0.4, fetch: int = 16):
    """
    Комбинированный поиск: score = alpha * sim(title) + (1-alpha) * sim(body)
    - k: сколько вернуть результатов
    - alpha: вес заголовка (0..1)
    - fetch: сколько кандидатов забираем из каждого индекса для объединения
    """
    # 1) эмбеддинги запроса для title/body (E5-подобные префиксы)
    q_title = model.encode([f"query_title: {query}"], normalize_embeddings=True).astype("float32")
    q_body  = model.encode([f"query_body: {query}"],  normalize_embeddings=True).astype("float32")

    # 2) быстрый предварительный топ по каждому индексу
    topn_t = min(max(fetch, k), idx_title.ntotal)
    topn_b = min(max(fetch, k), idx_body.ntotal)

    Dt, It = idx_title.search(q_title, topn_t)  # (1, topn_t)
    Db, Ib = idx_body.search(q_body,  topn_b)   # (1, topn_b)

    # 3) множество кандидатов из обоих топов
    cand_ids = sorted(set(It[0].tolist() + Ib[0].tolist()))
    if not cand_ids:
        return []

    # 4) точный пересчёт: достаём эмбеддинги кандидатов и считаем скоры
    #    (векторы нормированы -> dot == cosine)
    E_t = _reconstruct_batch(idx_title, cand_ids)   # (C, dim)
    E_b = _reconstruct_batch(idx_body,  cand_ids)   # (C, dim)

    s_title = (q_title @ E_t.T)[0]                  # (C,)
    s_body  = (q_body  @ E_b.T)[0]                  # (C,)
    scores  = alpha * s_title + (1.0 - alpha) * s_body

    # 5) ранжируем и собираем ответ
    order = np.argsort(-scores)[:k]
    results = []
    for r in order:
        idx = int(cand_ids[r])
        results.append({
            "score": float(scores[r]),
            **meta[idx]  # meta[idx] соответствует порядку индексации
        })
    return results

# пример
print(search("Как изменить вид на изометрический?", k=3, alpha=0.4))


In [ ]:
import os
import re
from typing import List, Dict, Optional
from openai import OpenAI
from openai import BadRequestError

# ====== НАСТРОЙКИ ПЕРЕКЛЮЧЕНИЯ ======
USE_LOCAL_LM: bool = bool(os.environ.get("USE_LOCAL_LM", "1") == "1")  # поставьте 1 чтобы включить локальный режим
LOCAL_LM_BASE_URL: str = os.environ.get("LOCAL_LM_BASE_URL", "http://localhost:1234/v1")
LOCAL_LM_API_KEY: str = os.environ.get("LOCAL_LM_API_KEY", "lm-studio")  # любой непустой токен
LOCAL_LM_MODEL: str = os.environ.get("LOCAL_LM_MODEL", "qwen/qwen3-8b")

# ====== КЛИЕНТЫ ======
# Облачный
client_cloud = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# Локальный (LM Studio OpenAI-compatible /v1/chat/completions)
client_local = OpenAI(base_url=LOCAL_LM_BASE_URL, api_key=LOCAL_LM_API_KEY)

SYSTEM_INSTRUCTIONS = (
    "You are a helpful assistant for a CAD app (SmartWOP). "
    "Answer ONLY using the provided context. If the answer is not in context, say you don't know. "
    "Prefer step-by-step, concise instructions. Keep tool/command names verbatim."
)

def build_context_blocks(hits: List[Dict], *, max_chars_per_block: int = 0) -> str:
    """
    Формирует контекст без [n]-меток:
    <TITLE>\n<BODY>\n\n...
    max_chars_per_block > 0 — мягкая обрезка каждого блока по символам (после тримминга).
    """
    blocks = []
    for h in hits:
        title = (h.get("title") or "").strip()
        body  = (h.get("body")  or "").strip()

        # опционально: компактная нормализация пробелов
        title = re.sub(r"\s+", " ", title)
        body  = re.sub(r"\s+", " ", body)

        block = f"{title}\n{body}" if title else body

        if max_chars_per_block and len(block) > max_chars_per_block:
            block = block[:max_chars_per_block].rstrip() + "…"

        blocks.append(block)

    return "\n\n".join(blocks)


# ====== ОБЩИЙ ВСПОМОГАТЕЛЬНЫЙ ШАГ: собираем user-промпт ======
def _build_user_prompt(user_query: str, context: str) -> str:
    return (
        f"User question (keep language of the user): {user_query}\n\n"
        f"=== CONTEXT START ===\n{context}\n=== CONTEXT END ===\n\n"
        f"Rules:\n"
        f"- Answer ONLY using the information from the context above.\n"
        f"- Do NOT mention block numbers or the word 'context'.\n"
        f"- Do NOT show your reasoning, inner thoughts, or explanations of how you found the answer.\n"
        f"- Provide only the final clear instructions or factual answer.\n"
        f"- Answer in the same language as the user question.\n"
        f"- If the answer is not in context, say 'I don't know'.\n"
    )

# ====== ВАШ ОБЛАЧНЫЙ ПУТЬ (responses API) — без изменений логики ======
def _create_response_safe_cloud(*, model: str, instructions: str, input_payload,
                                max_output_tokens: int = 400, temperature: Optional[float] = None):
    kwargs = dict(
        model=model,
        instructions=instructions,
        input=input_payload,
        max_output_tokens=max_output_tokens,
        store=False,
    )
    if temperature is not None:
        kwargs["temperature"] = temperature
        try:
            return client_cloud.responses.create(**kwargs)
        except BadRequestError as e:
            if "Unsupported parameter" in str(e) and "temperature" in str(e):
                kwargs.pop("temperature", None)
                return client_cloud.responses.create(**kwargs)
            raise
    else:
        return client_cloud.responses.create(**kwargs)

# ====== ЛОКАЛЬНЫЙ ПУТЬ (LM Studio /v1/chat/completions) ======
def _create_response_local_chat(*, model: str, system_text: str, user_text: str,
                                max_tokens: int = 400, temperature: Optional[float] = 0.2):
    # LM Studio поддерживает /v1/chat/completions; ключ можно любой непустой
    # https://lmstudio.ai/docs/app/api/endpoints/openai  (поддерживаются /v1/chat/completions, /v1/embeddings, /v1/completions)
    resp = client_local.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_text},
            {"role": "user",   "content": user_text},
        ],
        temperature=temperature,
        max_tokens=max_tokens,
        stream=False,
    )
    return resp

def ask_llm_with_rag(user_query: str, top_k: int = 5,
                     model: str = "gpt-4o-mini", temperature: Optional[float] = 0.2,
                     alpha: float = 0.4, fetch: int = 16) -> str:
    hits = search(user_query, k=top_k, alpha=alpha, fetch=fetch)
    context = build_context_blocks(hits)
    user_text = _build_user_prompt(user_query, context)

    # Если ничего не нашли — сразу честно говорим
    if not hits:
        fallback = (
            f"User question (keep language of the user): {user_query}\n\n"
            f"=== CONTEXT START ===\n(no results)\n=== CONTEXT END ===\n\n"
            f"Rules:\n"
            f"- Answer ONLY using the context blocks above.\n"
            f"- If the answer is not in context, say you don't know.\n"
            f"- Answer in the same language as the user question.\n"
        )
        user_text = fallback
    else:
        context = build_context_blocks(hits)
        user_text = _build_user_prompt(user_query, context)

    if USE_LOCAL_LM:
        # локальный режим: LM Studio (chat.completions)
        resp = _create_response_local_chat(
            model=os.environ.get("LOCAL_LM_MODEL", LOCAL_LM_MODEL),
            system_text=SYSTEM_INSTRUCTIONS,
            user_text=user_text,
            max_tokens=300,
            temperature=temperature,
        )
        return resp.choices[0].message.content

    else:
        # облачный режим (responses API)
        input_payload = [
            {
                "role": "user",
                "content": [
                    {"type": "input_text", "text": user_text}
                ],
            }
        ]
        resp = _create_response_safe_cloud(
            model=model,
            instructions=SYSTEM_INSTRUCTIONS,
            input_payload=input_payload,
            max_output_tokens=300,
            temperature=temperature,
        )
        return resp.output_text

# ====== Пример вызова ======
if __name__ == "__main__":
    q = "How to change view on isometric?"
    print(ask_llm_with_rag(q))


In [ ]:
# Jupyter cell: SmartWOP RAG UI (ipywidgets)
import traceback
from typing import List, Dict
from IPython.display import display, HTML, clear_output
import ipywidgets as w

# ====== UI widgets ======
title = w.HTML("<h2 style='margin:0'>SmartWOP Chat · RAG MVP</h2><p style='margin:4px 0 12px;color:#666'>Задай вопрос — получи ответ с цитированием источников [1], [2]…</p>")

q_input = w.Textarea(
    value="Wie benutze ich woodWOP Komponente?",
    placeholder="Введите вопрос (DE/EN)...",
    description="Вопрос:",
    layout=w.Layout(width="100%", height="80px")
)

model_dd = w.Dropdown(
    options=[
        ("gpt-4o-mini", "gpt-4o-mini"),
        ("gpt-5-nano", "gpt-5-nano"),
    ],
    value="gpt-4o-mini",
    description="Модель:",
    layout=w.Layout(width="300px")
)

topk_slider = w.IntSlider(
    value=5, min=1, max=10, step=1, description="Top-K:",
    continuous_update=False, readout=True, layout=w.Layout(width="300px")
)

temp_slider = w.FloatSlider(
    value=0.2, min=0.0, max=1.0, step=0.05, description="Temp:",
    continuous_update=False, readout=True, layout=w.Layout(width="300px")
)

full_toggle = w.Checkbox(value=False, description="Показывать полный текст статьи при раскрытии", indent=False)
stream_toggle = w.Checkbox(value=False, description="(опционально) Стриминг ответа в консоль", indent=False)

ask_btn = w.Button(
    description="Спросить",
    button_style="primary",
    icon="comment",
    layout=w.Layout(width="160px", height="38px")
)

status_out = w.Output(layout=w.Layout(border="1px solid #eee", padding="6px", max_height="80px", overflow_y="auto"))
ctx_out = w.Output(layout=w.Layout(border="1px solid #eee", padding="8px", max_height="250px", overflow_y="auto"))
ans_out = w.Output(layout=w.Layout(border="1px solid #eee", padding="12px"))

controls = w.HBox([model_dd, topk_slider, temp_slider])
toggles = w.HBox([full_toggle, stream_toggle])
go = w.HBox([ask_btn])

# ====== helpers ======
def _render_context(hits: List[Dict], full: bool = False) -> str:
    """Build HTML accordion with context hits."""
    if not hits:
        return "<div style='color:#999'>Контекст не найден.</div>"

    items_html = []
    for i, h in enumerate(hits, 1):
        title = h.get('title', f"Doc {i}")
        text = h.get('body', '')
        # Без f-строки, экранируем переносы заранее
        text_html = text.replace('\n', '<br>')
        items_html.append("""
        <details style='margin:8px 0;border:1px solid #eee;border-radius:8px;padding:6px;'>
          <summary style='cursor:pointer;font-weight:600'>
            [{}] {}<span style='color:#999;font-weight:400'> · score={:.2f}</span>
          </summary>
          <div style='margin-top:6px;line-height:1.45'>{}</div>
        </details>
        """.format(i, title, h.get('score', 0), text_html))
    return "".join(items_html)

def _render_answer(text: str) -> str:
    # лёгкая типографика с поддержкой жирного текста
    import re
    # Заменяем **текст** на <strong>текст</strong>
    safe = re.sub(r'\*\*(.*?)\*\*', r'<strong>\1</strong>', text)
    # Заменяем переносы строк на <br>
    safe = safe.replace("\n", "<br>")
    return "<div style='font-size:15px;line-height:1.55'>{}</div>".format(safe)

# ====== main handler ======
def on_ask_clicked(_):
    with status_out:
        clear_output()
        print("⏳ Запрашиваю контекст и ответ...")

    with ctx_out:
        clear_output()
    with ans_out:
        clear_output()

    user_q = q_input.value.strip()
    if not user_q:
        with status_out:
            clear_output()
            print("⚠️ Введите вопрос.")
        return

    try:
        # 1) Получаем контекст заранее, чтобы отрисовать
        #    Пытаемся вызвать ваш search(query, k=..., full=...) если он поддерживает full
        hits = []
        hits = search(user_q, k=topk_slider.value)

        hits = hits[:3]
        
        with ctx_out:
            clear_output()
            display(HTML("<b>Контекст:</b>"))
            display(HTML(_render_context(hits, full=full_toggle.value)))

        # 2) Ответ модели (ваша функция ask_llm_with_rag)
        #    Важно: пробрасываем top_k/model/temperature — чтобы UI реально влиял
        answer = ask_llm_with_rag(
            user_query=user_q,
            top_k=topk_slider.value,
            model=model_dd.value,
            temperature=temp_slider.value
        )

        with ans_out:
            clear_output()
            display(HTML("<b>Ответ:</b>"))
            display(HTML(_render_answer(answer)))

        with status_out:
            clear_output()
            print("✅ Готово.")

    except Exception as e:
        with status_out:
            clear_output()
            print("❌ Ошибка:")
            traceback.print_exc()

# bind
ask_btn.on_click(on_ask_clicked)

# ====== layout ======
display(
    w.VBox([
        title,
        q_input,
        controls,
        toggles,
        go,
        w.HTML("<hr style='border:none;border-top:1px solid #eee;margin:12px 0'>"),
        w.HTML("<b>Статус:</b>"),
        status_out,
        w.HTML("<b>Источники:</b>"),
        ctx_out,
        w.HTML("<b>Ответ:</b>"),
        ans_out
    ], layout=w.Layout(width="100%"))
)